# Shopping Mall

## 판매금액(spent)

---

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'NanumGothic'
rcParams['font.size'] = 10

### 파일 로드

In [ ]:
df = pd.read_excel('data/master.xlsx', sheet_name='order',encoding='cp949')
df

In [ ]:
df.columns

In [ ]:
df1 = df[['배송지','sex','spent','gubun','진행구분']].copy()
df1.head()

### 임직원 제외 & 구매확정

In [ ]:
df1['gubun'].unique()

In [ ]:
df['진행구분'].unique()

In [ ]:
df1 = df1.loc[(df1['gubun']=='일반') & ( (df1['진행구분']=='결제완료')  |
                                         (df1['진행구분']=='구매확정')   |
                                         (df1['진행구분']=='배송완료')   |
                                         (df1['진행구분']=='배송준비중') |
                                         (df1['진행구분']=='배송중')     |
                                         (df1['진행구분']=='주문완료')    )]

In [ ]:
len(df1)

### 성별 결측치 처리

In [ ]:
df1['sex'].value_counts()

In [ ]:
df1['sex'].isnull().sum()

In [ ]:
df1['sex'].fillna('U',inplace=True)

In [ ]:
df1['sex'].replace('-','U', inplace=True)

# apply 활용
#df1['sex'] = df1['sex'].apply(lambda x: 'U' if x == '-' else x)

# loc 활용
#df1.loc[df1['sex'] == '-', 'sex'] = 'U'

In [ ]:
df1['sex'].value_counts()

### 컬럼(지역)추가 from 배송지

In [ ]:
df1['배송지'].isnull().sum()

In [ ]:
def func1(x):
    if not isinstance(x,str): return '기타'
    if '서울'     in x: return '서울'
    if '세종'     in x: return '세종'
    if '제주'     in x: return '제주'
    if '경기'     in x: return '경기'
    if '인천'     in x: return '인천'
    if '대구'     in x: return '대구'
    if '전라북도' in x: return '전북'
    if '테헤란로' in x: return '서울'
    if '죽전'     in x: return '경기'
    if '라도무스' in x: return '기타'
    if '102*301'  in x: return '기타'
    if '호계e편한세상' in x: return '기타'
    return x.split()[0]

In [ ]:
df1['지역'] = df1['배송지'].apply(func1)

In [ ]:
df1['지역'].value_counts()

In [ ]:
df1.head()

### 지역별 판매금액 합계(비율)

In [ ]:
gp1 = df1.groupby('지역')

In [ ]:
df_sum = gp1.sum()

In [ ]:
df_sum

In [ ]:
# 판매금액 비율
df_sum['spent_ratio'] = df_sum.apply(lambda x: x/x.sum())

In [ ]:
df_sum.sort_values('spent_ratio', ascending=False)

In [ ]:
df_sum.loc[['서울','경기'],:]

### 지역별 인구 비율 

In [ ]:
# 인구현황
df_pop = pd.read_excel('data/pop_2017.xlsx', encoding='cp949')
df_pop

In [ ]:
# 인덱스 설정
df_pop.set_index('도시', inplace=True)

In [ ]:
# 컬럼 이름 변경
df_pop.rename({'인구':'pop'}, axis=1, inplace=True)

In [ ]:
# 도시 인구 비율
df_pop['pop_ratio'] = df_pop[['pop']].apply(lambda x: x/x.sum())

In [ ]:
df_pop.sort_index(inplace=True)
df_pop

In [ ]:
df_pop.sort_values('pop_ratio', ascending=False)

### 지역별 판매금액 & 인구 비율 비교 

In [ ]:
df_sp = pd.concat([df_sum, df_pop], axis=1, sort=False).drop('인구1000', axis=1)
df_sp

In [ ]:
df_sp.fillna(0,inplace=True)

In [ ]:
# 판매금액 비율 순서로 정렬
df_sp.sort_values('spent_ratio',ascending=False)

In [ ]:
# 인구 비율 순서로 정렬
df_sp.sort_values('pop_ratio',ascending=False)

In [ ]:
# <참고>

# 컬럼순서변경
#df_sum = df_sum[['spent','ratio','pop','pop_ratio']].copy()

# 컬럼이름변경
#df_sum.rename(columns = {'ratio':'spent_ratio'},inplace=True)

### 지역별, 성별 판매금액 합계(비율)

In [ ]:
gp2 = df1.groupby(['지역','sex'])

In [ ]:
gp2.sum()

In [ ]:
df_sex = gp2.sum().unstack()
df_sex

In [ ]:
# 결측치 처리
df_sex.fillna(0,inplace=True)

In [ ]:
# 계층적 인덱스(컬럼) 삭제
df_sex.columns=['F','M','U']
df_sex

In [ ]:
df_sex

In [ ]:
# 지역별, 성별 판매금액 비율
df_sex['F_ratio'] = df_sex[['F']].apply(lambda x: x/x.sum())
df_sex['M_ratio'] = df_sex[['M']].apply(lambda x: x/x.sum())
df_sex['U_ratio'] = df_sex[['U']].apply(lambda x: x/x.sum())

In [ ]:
df_sex.sort_index(inplace=True)

In [ ]:
df_sex.sort_values('F_ratio', ascending=False)

### 지역별 - (인구, 판매금액, 성별판매금액) 비율 비교

In [ ]:
df_result = pd.concat([df_sp,df_sex], axis=1)[['pop','pop_ratio','spent','spent_ratio',
                                                'F','M','U','F_ratio','M_ratio','U_ratio']]
df_result

### 결과 파일 저장

In [ ]:
df_result.to_csv('data/result/spent_ratio.csv', encoding='cp949')

### 그래프

In [ ]:
# 인구 비율, 판매금액 비율 비교
df_result[['pop_ratio','spent_ratio']].plot(kind='bar', figsize=(10,6))

In [ ]:
# 성별 판매 비율
df_result[['F_ratio','M_ratio','U_ratio']].plot(kind='bar', figsize=(10,6))

In [ ]:
# end of file